In [0]:
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

batch_size = 128
num_epochs = 50
learning_rate = 1e-3

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
train_set = DataLoader(datasets.CIFAR10('cifar',train=True, transform=transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
    #  transforms.RandomVerticalFlip(),
     transforms.RandomResizedCrop(size=(28,28), scale = (0.8, 1.0)),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ]
), download = True), batch_size = batch_size, shuffle = True)

val_set = DataLoader(datasets.CIFAR10('cifar',train=False, transform=transforms.Compose(
    [
    #  transforms.RandomHorizontalFlip(),
    #  transforms.RandomVerticalFlip(),
    #  transforms.RandomResizedCrop(size=(28,28), scale = (0.7, 1.0)),
     transforms.Resize((28,28)),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    ]
), download = True), batch_size = batch_size, shuffle = False)

print('Dataset is prepared')

Files already downloaded and verified
Files already downloaded and verified
Dataset is prepared


In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,64,3,stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64,128,3,stride = 2, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,256,3,stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256,256,3,stride = 1, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Flatten(),
            nn.Linear(1568*8,100),
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
    def forward(self, x):
        return self.net(x)

model = Model()
print(device)
model = model.to(device)

model.eval()
test_x = torch.rand((1,3,28,28)).to(device)
test_y = model(test_x).to(device)
print(test_y.shape)

crit = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience = 10, verbose = True)

print('Model and utils are prepared')

cuda
torch.Size([1, 10])
Model and utils are prepared


In [4]:
model.train()
for epoch in range(num_epochs):
    for idx, (x, y) in enumerate(train_set):
        x, y = x.to(device), y.to(device)
        prob = model(x)
        loss = crit(prob, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        period = 100
        if idx % period == period-1:
            print('Epoch: {}, IDX: {}, loss: {}'.format(epoch, idx+1, loss.item()))
    scheduler.step(loss)

Epoch: 0, IDX: 100, loss: 1.550129771232605
Epoch: 0, IDX: 200, loss: 1.3806085586547852
Epoch: 0, IDX: 300, loss: 1.2710720300674438
Epoch: 1, IDX: 100, loss: 1.050640344619751
Epoch: 1, IDX: 200, loss: 1.0692898035049438
Epoch: 1, IDX: 300, loss: 1.1701897382736206
Epoch: 2, IDX: 100, loss: 1.1836563348770142
Epoch: 2, IDX: 200, loss: 0.9632700681686401
Epoch: 2, IDX: 300, loss: 1.0556942224502563
Epoch: 3, IDX: 100, loss: 0.843894362449646
Epoch: 3, IDX: 200, loss: 1.1419159173965454
Epoch: 3, IDX: 300, loss: 0.9326617121696472
Epoch: 4, IDX: 100, loss: 0.9432413578033447
Epoch: 4, IDX: 200, loss: 0.9440531730651855
Epoch: 4, IDX: 300, loss: 1.1075613498687744
Epoch: 5, IDX: 100, loss: 0.9300475120544434
Epoch: 5, IDX: 200, loss: 0.9536318778991699
Epoch: 5, IDX: 300, loss: 0.9625968337059021
Epoch: 6, IDX: 100, loss: 0.7116847038269043
Epoch: 6, IDX: 200, loss: 0.8356720209121704
Epoch: 6, IDX: 300, loss: 0.9199984669685364
Epoch: 7, IDX: 100, loss: 0.713844895362854
Epoch: 7, IDX:

In [5]:
from google.colab import drive
drive.mount('/content/drive')
drive_folder = '/content/drive/My Drive/'
model_name = 'cifar10_model.pt'
# model.load_state_dict(torch.load(drive_folder+model_name))
torch.save(model.state_dict(),drive_folder+model_name)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
model.eval()
total_correct = 0
for x, y in val_set:
    x, y = x.to(device), y.to(device)
    prob = model(x)
    prid = torch.argmax(prob, dim = 1)
    total_correct += torch.sum(prid.eq(y)).float().item()

total = len(val_set.dataset)
acc = total_correct/total
print('total_correct: {}, total: {}, acc: {}'.format(total_correct, total, acc))

total_correct: 8392.0, total: 10000, acc: 0.8392
